In [ ]:
import os, sys
import pandas as pd
import numpy as np
import albumentations
import torch
from torch.utils.data import DataLoader

from sklearn import metrics
from sklearn.model_selection import train_test_split

In [1]:
# Import python files
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path+"/src")

import dataset
import engine
from model import get_alexnet, get_resnet, AlexNet

In [3]:
# Read data
data_path = "../data/"
df = pd.read_csv(os.path.join(data_path, "train.csv"))
# Get list of image files
images = df.ImageId.values.tolist()
images = [os.path.join(data_path, "train_png", i + ".png") for i in images]

In [4]:
# Check data
df.head()

,ImageId,EncodedPixels,target
0,1.2.276.0.7230010.3.1.4.8323329.13666.15178752...,557374 2 1015 8 1009 14 1002 20 997 26 990 32 ...,1
1,1.2.276.0.7230010.3.1.4.8323329.11028.15178752...,-1,0
2,1.2.276.0.7230010.3.1.4.8323329.10366.15178752...,514175 10 1008 29 994 30 993 32 991 33 990 34 ...,1
3,1.2.276.0.7230010.3.1.4.8323329.10016.15178752...,592184 33 976 58 956 73 941 88 926 102 917 109...,1
4,1.2.276.0.7230010.3.1.4.8323329.11444.15178752...,-1,0


In [5]:
images[:5]

['../data/train_png/1.2.276.0.7230010.3.1.4.8323329.13666.1517875247.117800.png',
 '../data/train_png/1.2.276.0.7230010.3.1.4.8323329.11028.1517875229.983789.png',
 '../data/train_png/1.2.276.0.7230010.3.1.4.8323329.10366.1517875223.393986.png',
 '../data/train_png/1.2.276.0.7230010.3.1.4.8323329.10016.1517875220.992175.png',
 '../data/train_png/1.2.276.0.7230010.3.1.4.8323329.11444.1517875232.977506.png']

In [6]:
# Set device and epochs
device = "cuda"
epochs = 10

targets =df.target.values
# model = get_alexnet(pretrained=True)
# model = get_resnet(pretrained=True)
model = AlexNet()
model.to(device)
model

AlexNet(
  (conv1): Conv2d(3, 96, kernel_size=(11, 11), stride=(4, 4))
  (pool1): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(96, 256, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (pool2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv3): Conv2d(256, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv4): Conv2d(384, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv5): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool3): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=9216, out_features=4096, bias=True)
  (dropout1): Dropout(p=0.5, inplace=False)
  (fc2): Linear(in_features=4096, out_features=4096, bias=True)
  (last_linear): Sequential(
    (0): BatchNorm1d(4096, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (1): Dropout(p=0.25, inplace=False)
    (2): Linear(in_featu

In [7]:
# Image augmentation
# mean and std of RGB channels
mean = (0.485, 0.456, 0.406)
std = (0.229, 0.224, 0.225)
aug = albumentations.Compose(
    [albumentations.Normalize(mean, std, max_pixel_value=255.0, always_apply=True)]
)
# aug = None

In [8]:
# prepare training and validation datasets
train_images, valid_images, train_targets, valid_targets = train_test_split(
    images, targets, stratify=targets, random_state=42
)

train_dataset = dataset.ClassificationDataset(
    image_paths=train_images, targets=train_targets, 
    resize=(227, 227), augmentations=aug
)
valid_dataset = dataset.ClassificationDataset(
    image_paths=train_images, targets=train_targets,
    resize=(227, 227), augmentations=aug
)

In [10]:
type(train_images), type(valid_images), type(train_dataset)

(list, list, dataset.ClassificationDataset)

In [11]:
 train_images[:5]

['../data/train_png/1.2.276.0.7230010.3.1.4.8323329.13152.1517875243.898511.png',
 '../data/train_png/1.2.276.0.7230010.3.1.4.8323329.1858.1517875169.946702.png',
 '../data/train_png/1.2.276.0.7230010.3.1.4.8323329.5369.1517875187.802444.png',
 '../data/train_png/1.2.276.0.7230010.3.1.4.8323329.12299.1517875238.450255.png',
 '../data/train_png/1.2.276.0.7230010.3.1.4.8323329.32482.1517875160.802571.png']

In [15]:
train_dataset[1]["image"].shape

torch.Size([3, 227, 227])

In [16]:
data = train_dataset[1]
inputs = data["image"]
targets = data["targets"]
# Move data into device
inputs = inputs.to(device, dtype=torch.float)
targets = targets.to(device, dtype=torch.float)


In [21]:
inputs.shape, targets.shape, targets

(torch.Size([3, 227, 227]), torch.Size([]), tensor(0., device='cuda:0'))

In [24]:
targets.view(-1,1)

tensor([[0.]], device='cuda:0')

In [9]:
# Get data loader and training
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True, num_workers=4)
valid_loader = DataLoader(valid_dataset, batch_size=16, shuffle=True, num_workers=4)

optimizer = torch.optim.Adam(model.parameters(), lr=5e-4)

for epoch in range(epochs):
    engine.train(train_loader, model, optimizer, device)
    preds, valid_targets = engine.evaluate(valid_loader, model, device)
    auc = metrics.roc_auc_score(valid_targets, preds)
    print(f"Epoch={epoch}, Valid AUC={auc}")

Epoch=0, Valid AUC=0.5261574094094008
Epoch=1, Valid AUC=0.5991361478797208
Epoch=2, Valid AUC=0.6209951524533948
Epoch=3, Valid AUC=0.6268789107938992
Epoch=4, Valid AUC=0.6926146175223746
Epoch=5, Valid AUC=0.6179618772819722
Epoch=6, Valid AUC=0.7173027540061088
Epoch=7, Valid AUC=0.6845520848198133
Epoch=8, Valid AUC=0.7026812857025483
Epoch=9, Valid AUC=0.6444200962013857
